In [1]:
import pydicom as pydcm
import matplotlib.pyplot as plt
import pandas as pd
import sys
from pydicom.datadict import dictionary_VR
from pydicom import dcmread,Dataset
from pydicom.filewriter import dcmwrite
import numpy as np
#import cv2
import random
import time
import datetime


from google.cloud import storage
from io import BytesIO
client = storage.Client()

In [2]:
def blob_to_image(bucket, blob_name):
    blob = bucket.get_blob(blob_name)
    blobstring = blob.download_as_string()
    dicomfile = BytesIO(blobstring)
    dicomread = dcmread(dicomfile)
    #plt.imshow(dicomread.pixel_array)
    return dicomread

def img_diff(raw, deid):
    diff = raw.pixel_array-deid.pixel_array
    result = np.count_nonzero(diff)
    return(result)    

def find(list,thing): 
    "finds indices of something in a list"
    return [i for i, j in enumerate(list) if j == thing]


def blob_to_df(filelist, bucket):
    SOP = []
    patient = []
    study = []
    series = []
    DICOM = []
    for i in range(len(filelist)):
        dicom = blob_to_image(bucket, filelist[i])
        DICOM.append(dicom)
        patient.append(dicom.PatientID)
        study.append(dicom.StudyInstanceUID)
        series.append(dicom.SeriesInstanceUID)
        SOP.append(dicom.SOPInstanceUID)
    
    d = {'dicom': DICOM, 'patientID': patient, 'studyID': study, 'seriesID': series, 
         'SOPID': SOP}
    df = pd.DataFrame(d)
    return(df)


In [3]:
bucket_in = client.get_bucket('midi-import')

blobs_in = bucket_in.list_blobs(prefix='ds1_1-23/')
filelist_in = [item.name for item in blobs_in] #extracting file names from blobs

df = blob_to_df(filelist_in[1:], bucket_in)
#df_in = blob_to_df(filelist_in[1:], bucket_in)

/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '2.5.428.0.3.5852292.6.070.1048487533109005323'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '2.5.428.0.3.5852292.6.070.1281941386527526536'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '2.5.428.0.3.5852292.6.070.2964773742202960509'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '2.5

In [4]:
import re

def filterLength(n):
    if len(n)>=6:
        return True
    else:
        return False


In [ ]:
#manual code final 
file = 0
vrs = ['LO', 'SH', 'LT', 'ST', 'UT','IS']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []

#loop through each dicom file that was imported
for i in range(len(df)):
    file = df.iloc[i][0]
    #loop through each element in the dicom file
    for elem in file.iterall():
        #only looks at elements with VRs 'LO', 'SH', 'LT', 'ST', 'UT','IS'
        if elem.VR in vrs:
            #ignores elements without numeric characters
            if str(elem.value).isalpha() == False:
                str1 = elem.value
                try:
                    #looks for strings of continuous numeric values in the element
                    x = re.findall('[0-9]+', str1)
                    #filters the list of continuous numeric values to strings of length 6 or more
                    x2 = list(filter(filterLength, x))
                    if len(x2) != 0:
                        #appends the information of the elements where all the previous conditions are met
                        patient.append(df.iloc[i][1])
                        study.append(df.iloc[i][2])
                        series.append(df.iloc[i][3])
                        SOP.append(df.iloc[i][4])
                        element.append(str(elem))
                        tag.append(elem.name)
                        vr.append(elem.VR)
                        value.append(elem.value)
                except:
                    print(elem)
                                      
                        
    

In [7]:
#creates a dataframe with all of the flagged elements
d = {'patient':patient, 'study':study, 'series':series, 'SOP':SOP, 'tag':tag, 'vr':vr, 'element':element, 'value':value}
flagged = pd.DataFrame(d)

In [8]:
#removes certain tags from the flagged elements
flagged2 = flagged[flagged['tag']!='Code Value']
flagged2 = flagged2[flagged2['tag']!='Patient ID']
flagged2 = flagged2[flagged2['tag']!='Study ID']
flagged2 = flagged2[flagged2['tag']!='Clinical Trial Subject ID']
flagged2 = flagged2[flagged2['tag']!='Accession Number']

In [9]:
#takes the unique errors of the flagged elements
flagged2['tag'].unique()

array(['Additional Patient History', 'Patient Comments',
       'Actual Frame Duration', 'Series Description', 'Study Comments',
       'Series Number', 'Scheduled Procedure Step Description',
       'Study Description', 'Private Creator', 'Protocol Name',
       'Requested Procedure Comments', 'Requested Procedure Description',
       'Imaging Service Request Comments',
       'Attenuation Correction Method', 'Acquisition Comments',
       'Private tag data', 'Image Comments', 'Text Value',
       'Medical Alerts'], dtype=object)

In [11]:
unique_flagged = flagged2.drop_duplicates(subset=['study','element'])

In [25]:
unique_flagged.to_csv('allflaggedtest.csv')

#the csv is then reviewed manually, and then dates of re-formatted and the patient IDs are removed by hand